## 패키지 불러오기 

In [ ]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import matplotlib.pyplot as plt # 데이터 시각화

from datetime import datetime, timedelta # 시간 데이터 처리
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
from tqdm import tqdm_notebook
%matplotlib inline

## 데이터 불러오기 

In [ ]:
path = 'C:/Users/choco/Desktop/Github/Dacon/Energy Bigdata Utilization/'
train = pd.read_csv(path + "input/train2.csv")
test = pd.read_csv(path + "input/test2.csv")
submission = pd.read_csv(path + "input/submission_1002.csv")
new_df = pd.read_csv(path + 'input/new_df_test.csv')

In [ ]:
new_df=new_df.set_index('time') # time을 인덱스로 저장합니다.
new_df.head()

## Prophet 

In [ ]:
from fbprophet import Prophet

def get_prophet_model(data):
    agg = {}
    for key in tqdm_notebook(new_df['place_id'].unique()):
        temp = new_df[new_df['place_id'] == key]
        temp_1h = temp.resample('1h').sum().reset_index()[['time','target']].rename(columns={'time':'ds','target':'y'})
        temp_1day = temp.resample('D').sum().reset_index()[['time','target']].rename(columns={'time':'ds','target':'y'})

        # 시간별 예측 
        m = Prophet(daily_seasonality=True)
        m.fit(temp_1h) 
        future = m.make_future_dataframe(periods=24, freq = 'H')
        fcst = m.predict(future)
        fcst = fcst.loc[fcst['ds'] >= pd.datetime(2018,7,1)][['ds','yhat']].T


        a = pd.DataFrame() # a라는 데이터프레임에 예측값을 정리합니다.

        for i in range(24):
            a['X2018_7_1_'+str(i+1)+'h']=[fcst.iloc[1, i]] # column명을 submission 형태에 맞게 지정합니다.

        m = Prophet(daily_seasonality=True)
        m.fit(temp_1day)
        future = m.make_future_dataframe(periods=10)
        fcst = m.predict(future)
        fcst = fcst.loc[fcst['ds'] >= pd.datetime(2018,7,1)][['ds','yhat']].T

        for i in range(10):
            a['X2018_7_'+str(i+1)+'_d']=[fcst.iloc[1, i]] # column명을 submission 형태에 맞게 지정합니다.

        m = Prophet(daily_seasonality=True)
        m.fit(temp_1day)
        future = m.make_future_dataframe(periods = 5, freq = 'M')
        fcst = m.predict(future)
        fcst = fcst.loc[fcst['ds'] >= pd.datetime(2018,7,1)][['ds','yhat']].T

        for i in range(5):
            a['X2018_'+str(i+7)+'_m']=[fcst.iloc[1, i]] # column명을 submission 형태에 맞게 지정합니다.

        a['meter_id'] = key 
        agg[key] = a[submission.columns.tolist()]
    print('---- Modeling Done ----')
    return agg 

In [ ]:
agg = get_prophet_model(new_df)

In [ ]:
output1 = pd.concat(agg, ignore_index=False)
output2 = output1.reset_index().drop(['level_0','level_1'], axis=1)
output2['id'] = output2['meter_id'].str.replace('X','').astype(int)
output2 =  output2.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)
output2.to_csv(path + '/Output/sub_baseline_prophet.csv', index=False)